# Importing the Data and The Libraries

In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import decomposition


from sklearn.decomposition import KernelPCA, PCA

df = pd.read_csv("HumidityDataset.csv")

C:\Users\sherwin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,2,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Dropping all the libraires and selecting only rows that start after the 2010

In [2]:
df.drop(["longitude","latitude", "WaveHeight", "WavePeriod", "MeanWaveDirection", "Hmax","QC_Flag"],inplace = True, axis = 1)
df = df.iloc[331371:]
buoy_ident = { 'M2':1 , 'M3': 2, 'M4':3, 'M5': 4, 'M6': 5}
# df.station_id.nunique()

In [3]:
# Selecting only the values of rows that have buoys ident as M2 to M6
df = df.loc[df.station_id.isin(buoy_ident.keys()) ]
df = df.drop(["time"], axis = 1)
# df.station_id.unique()

In [4]:
df = df.replace({ 'station_id': buoy_ident})
df = df.dropna(axis = 1, how='all')
df.reset_index(inplace = True)
df.dropna(inplace = True, how = 'all')
# df.head()

In [5]:
df = df.dropna()
df.drop('index', axis = 1, inplace = True)
df.head()

# df.reset_index(inplace= True)
# df.isna()
# drop('index',axis = 1, inplace = True)

,station_id,AtmosphericPressure,WindDirection,WindSpeed,Gust,AirTemperature,DewPoint,SeaTemperature,RelativeHumidity
0,1,1007.2,50,17,25,5.5,-1.1,10.3,62
1,5,1015.8,20,18,33,7,0.9,11.4,65
3,3,1011.2,10,12,20,4.7,1.1,10.2,78
4,2,1009.4,30,17,25,5.7,-1.2,10.4,61
5,2,1009,20,18,25,5.3,-0.3,10.4,67


In [6]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(df.iloc[:,:-1].values,df.iloc[:,-1].values, test_size=0.2)
print(X_train1.shape)

class CustomScaler():
    
        def __init__(self, X_Scale, y_Scale ) :
        
            self.X_Scale =   X_Scale 
            self.y_Scale =   y_Scale
            
            self.x_num =     [np.mean(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            self.x_maxs =    [np.max(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            self.x_mins =    [np.min(self.X_Scale[:,i]) for i in range(self.X_Scale.shape[1])]
            
            self.y_num =     np.mean(self.y_Scale)
            self.y_max_min = np.max(self.y_Scale) - np.min(self.y_Scale)

        def scaleX(self, x_value):
            x = x_value.copy()
            for i in range(x.shape[1]):
                
                x[:,i]= (x[:,i] - self.x_num[i])/(self.x_maxs[i]-self.x_mins[i])
            return x

        def inverseScaleX(self, x_value):
            x = x_value.copy()
            
            for i in range(x.shape[1]):
                x[:,i]= (x[:,i] * (self.x_maxs[i]-self.x_mins[i])) + self.x_num[i]
            return x



        

        def scaleY(self, y_value):
            y = y_value.copy()
            ys = (y - self.y_num)/(self.y_max_min)
            return ys

        def inverseScaleY(self, y_value):
            y = y_value.copy()
            ys = (y * self.y_max_min) + self.y_num
            return ys


        



custom_scaler = CustomScaler(X_train1,y_train1)
    
X_train = custom_scaler.scaleX(X_train1.copy())
y_train = custom_scaler.scaleY(y_train1.copy())

X_train = np.asarray(X_train).astype('float32')
y_train = np.asarray(y_train).astype('float32')
transform = None


print(X_train[-70000: -1])

print(y_train1.shape)


(68427, 8)
[[ 0.24974425 -0.00056802 -0.04797773 ...  0.14099914  0.19979288
   0.12685025]
 [ 0.24974425  0.04002238  0.3964667  ...  0.17509004  0.02975238
   0.21613596]
 [-0.50025576  0.04002238  0.4242445  ... -0.02377359 -0.01073345
  -0.08743546]
 ...
 [ 0.24974425 -0.01606617 -0.02019995 ...  0.15804459  0.13096696
   0.13577883]
 [-0.50025576  0.06363862  0.3131334  ...  0.13531731  0.14716129
   0.15363596]
 [ 0.24974425  0.04076039 -0.04797773 ...  0.08418095  0.10667546
   0.07327882]]
(68427,)


In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

clf = RandomForestRegressor(n_estimators=1000, random_state=0, n_jobs=-1)

clf.fit(X_train, y_train)


RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)

In [18]:
x_columns = df.columns[0:-1]

for feature in zip(x_columns, clf.feature_importances_):
    print(feature)
    
sfm = SelectFromModel(clf, threshold=0.15)

# Train the selector
sfm.fit(X_train, y_train)

for feature_list_index in sfm.get_support(indices=True):
    print(x_columns[feature_list_index])

('station_id', 0.00011547026245210912)
('AtmosphericPressure', 0.00016945066588258196)
('WindDirection', 0.000124555107387325)
('WindSpeed', 0.00010133218437444298)
('Gust', 0.00010008541005791994)
('AirTemperature', 0.20218923334652647)
('DewPoint', 0.7133706926345369)
('SeaTemperature', 0.0838291803887822)
AirTemperature
DewPoint


In [17]:
df.columns[0:-1]

Index(['station_id', 'AtmosphericPressure', 'WindDirection', 'WindSpeed',
       'Gust', 'AirTemperature', 'DewPoint', 'SeaTemperature'],
      dtype='object')

In [11]:

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
def model1():    
    model = keras.Sequential()
    model.add(Dense(10, activation = "relu",input_dim = X_train.shape[1], name = "layer1"))
    model.add(Dense(20, activation = "relu", name = "layer2"))
    model.add(Dense(20, activation = "relu", name = "layer5"))
    model.add(Dense(20, activation = "relu", name = "layer6"))

    model.add(Dense(1, kernel_initializer='normal', name = "layer7"))
    model.compile(loss="mean_squared_error", optimizer='SGD')
    return model

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

estimator = KerasRegressor(build_fn=model1, epochs=1000, batch_size=4096, verbose=1,callbacks=[es])
history=estimator.fit(np.asarray(X_train).astype('float32'),np.asarray(y_train).astype('float32'))



#history = model.fit(x_pca, df.iloc[:, -1], batch_size=batch_size, epochs=20, validation_split=0.1)
# estimator = KerasRegressor(build_fn=model, epochs=100, batch_size=batch_size, verbose=0)
# history = model.fit(np.asarray(x).astype('float32'),np.asarray(y).astype('float32') , epochs=150, batch_size=50,  verbose=1, validation_split=0.3)
# kfold = KFold(n_splits=5)
# results = cross_val_score(estimator,np.asarray(x).astype('float32'), np.asarray(y).astype('float32'), cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
if transform:
    y_pred = estimator.predict(transformer.transform(custom_scaler.scaleX(np.asarray(X_test1).astype('float32'))))
else :
    y_pred = estimator.predict((custom_scaler.scaleX(np.asarray(X_test1).astype('float32'))))

# y_pred = max_min*y_pred + y_min
print(mean_squared_error(custom_scaler.inverseScaleY(y_pred), y_test1))
y_test1

In [ ]:
np.column_stack((y_test1,custom_scaler.inverseScaleY(y_pred)))

In [ ]:
predTest = scaler.inverse_transform(np.column_stack((X_test, y_pred)))
ogTest = scaler.inverse_transform(np.column_stack((X_test, y_test)))
np.column_stack((predTest[:, -1], ogTest[:, -1]))